In [26]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import addfips


In [27]:
df = pd.DataFrame({"State":[], "Interpersonal Deaths":[],"Sucide Rate":[], "Obesity":[],"Asthma":[],"Cancer":[], "Drug OD":[],"Smoking": [], "Alcohol":[]})
df

,State,Interpersonal Deaths,Sucide Rate,Obesity,Asthma,Cancer,Drug OD,Smoking,Alcohol


In [28]:
def clean_state_data(df):
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code"])
    df = df[df["Crude Rate"] != "Unreliable"]
    df["Rate"] = (df["Deaths"]/df["Population"])*100000
    df["Rate"] = df["Rate"].round(2)
    
    df = df.rename({"ICD-10 113 Cause List":"Cause"},axis=1)
    df = df.sort_values(by=["Year", "Rate"], ascending=False)
    return df
    
state_df = pd.read_csv(r"data/Life/Deaths/2018CuaseOfDeath.txt", delimiter="	")
state_df = clean_state_data(state_df)
state_df.sample(3)

,Notes,State,State Code,Year,Year Code,Cause,ICD-10 113 Cause List Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Rate
3060,NaN,New Hampshire,33.0,2018.0,2018.0,Other forms of chronic ischemic heart disease ...,GR113-061,1110.0,1356458.0,81.8,77.0,86.6,81.83
814,NaN,Delaware,10.0,2018.0,2018.0,"Transport accidents (V01-V99,Y85)",GR113-113,130.0,967171.0,13.4,11.1,15.8,13.44
1224,NaN,Hawaii,15.0,2018.0,2018.0,Intentional self-harm (suicide) by discharge o...,GR113-125,30.0,1420491.0,2.1,1.4,3.0,2.11


In [29]:
state_df[state_df["Cause"]=="#Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)"].sample(3)

,Notes,State,State Code,Year,Year Code,Cause,ICD-10 113 Cause List Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Rate
4466,NaN,Tennessee,47.0,2018.0,2018.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,1161.0,6770010.0,17.1,16.2,18.1,17.15
904,NaN,District of Columbia,11.0,2018.0,2018.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,54.0,702455.0,7.7,5.8,10.0,7.69
2811,NaN,Montana,30.0,2018.0,2018.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,265.0,1062305.0,24.9,21.9,27.9,24.95


In [71]:
column_rename ={"#Accidents (unintentional injuries) (V01-X59,Y85-Y86)": "Accidents",
    "#Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)": "Self Harm",
    "#Diseases of heart (I00-I09,I11,I13,I20-I51)":"Heart Disease",
    "#Malignant neoplasms (C00-C97)": "Cancer",
    "#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)": "Assault",
    "#Certain conditions originating in the perinatal period (P00-P96)":"Child Birth"}

In [73]:
state_df

,Notes,State,State Code,Year,Year Code,Cause,ICD-10 113 Cause List Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Rate
38,NaN,Alabama,1.0,2018.0,2018.0,Major cardiovascular diseases (I00-I78),GR113-053,17523.0,4887871.0,358.5,353.2,363.8,358.50
5041,NaN,West Virginia,54.0,2018.0,2018.0,Major cardiovascular diseases (I00-I78),GR113-053,6465.0,1805832.0,358.0,349.3,366.7,358.01
343,NaN,Arkansas,5.0,2018.0,2018.0,Major cardiovascular diseases (I00-I78),GR113-053,10416.0,3013825.0,345.6,339.0,352.2,345.61
2544,NaN,Mississippi,28.0,2018.0,2018.0,Major cardiovascular diseases (I00-I78),GR113-053,10306.0,2986530.0,345.1,338.4,351.7,345.08
3791,NaN,Oklahoma,40.0,2018.0,2018.0,Major cardiovascular diseases (I00-I78),GR113-053,13213.0,3943079.0,335.1,329.4,340.8,335.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4556,NaN,Texas,48.0,2018.0,2018.0,#Diseases of appendix (K35-K38),GR113-091,20.0,28701845.0,0.1,0.0,0.1,0.07
415,NaN,California,6.0,2018.0,2018.0,Other tuberculosis (A17-A19),GR113-006,25.0,39557045.0,0.1,0.0,0.1,0.06
481,NaN,California,6.0,2018.0,2018.0,#Acute bronchitis and bronchiolitis (J20-J21),GR113-080,24.0,39557045.0,0.1,0.0,0.1,0.06
499,NaN,California,6.0,2018.0,2018.0,"Chronic glomerulonephritis, nephritis and neph...",GR113-099,21.0,39557045.0,0.1,0.0,0.1,0.05


In [72]:
pivot_state = state_df.pivot(index="State", columns="Cause", values="Crude Rate")
pivot_state = pivot_state.reset_index().rename(column_rename,axis=1)
final_cols = list(column_rename.values())
final_cols.append("State")
pivot_state =pivot_state[final_cols]
pivot_state.set_index("State").reset_index()

Cause,State,Accidents,Self Harm,Heart Disease,Cancer,Assault,Child Birth
0,Alabama,54.9,16.8,275.6,217.5,11.6,3.9
1,Alaska,54.2,25.0,110.5,129.8,7.6,NaN
2,Arizona,59.3,20.1,173.7,168.9,5.9,3.1
3,Arkansas,50.8,18.4,271.1,215.4,8.8,4.0
4,California,35.8,11.4,158.1,151.6,4.8,2.5
5,Colorado,53.5,22.5,129.4,137.2,4.6,2.8
6,Connecticut,57.5,11.7,201.7,181.2,2.6,2.3
7,Delaware,68.4,11.7,209.9,217.2,5.9,2.9
8,District of Columbia,56.2,7.7,184.4,152.3,20.5,4.4
9,Florida,59.5,16.7,220.8,212.6,6.2,3.3
